In [1]:
import pandas as pd
from datetime import datetime
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import openpyxl

from openpyxl.styles.alignment import Alignment
from openpyxl.styles import Font
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from openpyxl.styles import Border, Side
from openpyxl.drawing.image import Image

#Define all the functions we have previously written

def create_yield_table(data, date):
    date2 = date.replace(year=date.year-1)
    date3 = date.replace(year=date.year-5)
    
    curve1 = data.loc[date]
    curve2 = data.loc[date2]
    curve3 = data.loc[date3]
    
    table = pd.concat([curve1, curve2, curve3], axis=1)
    table.columns = ["Current", "1 Year Ago Curve", "5 Years Ago Curve"]
    table["1 Year Change"] = table["Current"] - table["1 Year Ago Curve"]
    table["5 Year Change"] = table["Current"] - table["5 Years Ago Curve"]
    table = table.reindex(columns=['Current', '1 Year Ago Curve','1 Year Change', '5 Years Ago Curve','5 Year Change'])
    return table

def create_title(ws, area, text):
    ws.merge_cells(area)
    #Given an area such as A1:A5, we want to first isolate the first cell
    first_cell = area.split(":")[0]
    #And then find the numerical row and column associated
    first_cell = coordinate_from_string(first_cell)
    col = column_index_from_string(first_cell[0])
    row = first_cell[1]
    
    #Change the text of the merged cell to whatever was passed as input
    ws.cell(row=row, column=col).value = text
    #Change the font of the title
    font = font = Font(size=16,
                       bold=True,
                      color="283747")
    ws.cell(row=row, column=col).font = font
    #Change alignment to be centered
    ws.cell(row=row, column=col).alignment = Alignment(horizontal="center")
    
def convert_to_year_frac(label):
    if "Mo" in label:
        frac = label.replace(' Mo','')
        frac = float(frac)
        frac = frac/12
    else:
        frac = label.replace(' Yr','')
        frac = float(frac)
    return frac

def write_to_excel(ws, start_row, start_column, df, index=True, columns = True):
    #If we want to include the index, reset the index on the dataframe first
    if index:
        write_data = df.reset_index().values.tolist()
    else:
        write_data = df.values.tolist()
    
    #If we want to include columns we need to add the columns as the first in the nested lists
    if columns:
        #If we already added an index, then we are going to want also include a blank space since
        #we don't want a label above the index
        if index:
            write_data = [[""]+list(df.columns)]+write_data
        else:
            write_data = [list(df.columns)]+write_data
    
    #For every value iterate through finding the location to place it in
    for row in range(len(write_data)):
        for column in range(len(write_data[0])):
            ws.cell(row=start_row+row,column=start_column+column).value = write_data[row][column]
def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

def adjust_columns(ws,start,values):
    for i, width in enumerate(values):
        ws.column_dimensions[get_column_letter(start+i)].width = width

def value_to_red_green(value):
    if value > 0:
        return (int(255-value*255), 255, int(255-value*255))
    else:
        value = abs(value)
        return (255, int(255-value*255), int(255-value*255))
    
def create_color_scale(returns):
    max_abs_return = abs(returns).max().max()
    scale = returns / max_abs_return
    
    #Map the conversion to rgb
    scale = scale.applymap(value_to_red_green)
    
    #Map rgb to hex
    scale = scale.applymap(rgb_to_hex)
    
    return scale

def red_green_map(ws, returns, start_row, start_column):
    color_scale = create_color_scale(returns.fillna(0))
    
    for row in range(returns.shape[0]):
        for column in range(returns.shape[1]):
            color = PatternFill(start_color=color_scale.iloc[row, column],
                   end_color=color_scale.iloc[row, column],
                   fill_type='solid')
            
            ws.cell(row=start_row+row,column=start_column+column).fill = color

def bold_cells_line(ws, start_row, start_column, length, vertical=True):
    for i in range(length):
        if vertical:
            cell = ws.cell(row=start_row+i,column=start_column)
        else:
            cell = ws.cell(row=start_row,column=start_column+i)
        font = cell.font.copy()
        font.bold = True
        cell.font = font

def create_axis_labels(ws, real_estate_returns):

    ws.merge_cells(start_row=4, start_column=1, end_row=5+real_estate_returns.shape[0]-1, end_column=1)
    ws.cell(row=4,column=1).value = "Year"
    #Text rotation is an option to rotate the text
    ws.cell(row=4, column=1).alignment = Alignment(horizontal="center", vertical="center", textRotation=90)
    ws.cell(row=4, column=1).font = Font(size=16, bold=True, color="283747")

    ws.merge_cells("B3:F3")
    ws.cell(row=3,column=2).value = "Quarter"
    ws.cell(row=3, column=2).font = Font(size=16, bold=True, color="283747")
    ws.cell(row=3, column=2).alignment = Alignment(horizontal="center", vertical="center")
    
def create_number_formatting(ws, start_row, start_column, vertical_length, horizontal_length,num_format):
    for row in range(vertical_length):
        for column in range(horizontal_length):
            ws.cell(row=start_row+row,column=start_column+column).number_format = num_format

def create_border(ws, start_row, start_column, vertical_length, horizontal_length):
    border = Border(left=Side(border_style='thin'),
         right=Side(border_style='thin'),
         top=Side(border_style='thin'),
         bottom=Side(border_style='thin'))
    for row in range(vertical_length):
        for column in range(horizontal_length):
            ws.cell(row=start_row+row,column=start_column+column).border = border
            
def write_treasury_sheet(ws, yield_curve_table):
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_columns(ws,2,[15]*len(yield_curve_table.columns))
    
    #Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    #Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
    #Add borders
    create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])
    
    #Get rid of gridlines
    ws.sheet_view.showGridLines = False
    
    #Add yield curve graphs
    img = Image('Images/Treasury Yield Curves.png')
    ws.add_image(img, 'H1')
    
def create_real_estate_report(ws, real_estate_returns, image_name):
    #Write the table
    write_to_excel(ws, 4, 2, real_estate_returns)
    
    #Create the axis labels
    create_axis_labels(ws, real_estate_returns)
    
    #Create percentage number formatting
    create_number_formatting(ws, 5, 3, real_estate_returns.shape[0], real_estate_returns.shape[1],"0.0%")
    
    #Map the red-green color map
    red_green_map(ws, real_estate_returns, 5, 3)
    
    #Add image
    img = Image(image_name)
    ws.add_image(img, 'H5')

In [2]:
#Create a function to handle creation of yield curve data
def prepare_data_yield_curve():
    yield_curve = pd.read_csv("Data/Yield Curve.csv",index_col=0)
    yield_curve.index = pd.to_datetime(yield_curve.index)

    yield_curve = yield_curve.drop(columns="2 Mo")
    yield_curve = yield_curve.dropna()
    date_range = pd.date_range(yield_curve.index.min(), yield_curve.index.max())
    yield_curve = yield_curve.reindex(index=date_range)
    yield_curve = yield_curve.fillna(method='ffill')
    return yield_curve
    
    
yield_curve = prepare_data_yield_curve()
print(yield_curve)

            1 Mo  3 Mo  6 Mo  1 Yr  2 Yr  3 Yr  5 Yr  7 Yr  10 Yr  20 Yr  \
2001-07-31  3.67  3.54  3.47  3.53  3.79  4.06  4.57  4.86   5.07   5.61   
2001-08-01  3.65  3.53  3.47  3.56  3.83  4.09  4.62  4.90   5.11   5.63   
2001-08-02  3.65  3.53  3.46  3.57  3.89  4.17  4.69  4.97   5.17   5.68   
2001-08-03  3.63  3.52  3.47  3.57  3.91  4.22  4.72  4.99   5.20   5.70   
2001-08-04  3.63  3.52  3.47  3.57  3.91  4.22  4.72  4.99   5.20   5.70   
2001-08-05  3.63  3.52  3.47  3.57  3.91  4.22  4.72  4.99   5.20   5.70   
2001-08-06  3.62  3.52  3.47  3.56  3.88  4.17  4.71  4.99   5.19   5.70   
2001-08-07  3.63  3.52  3.47  3.56  3.90  4.19  4.72  5.00   5.20   5.71   
2001-08-08  3.61  3.49  3.41  3.46  3.77  4.05  4.61  4.87   4.99   5.61   
2001-08-09  3.61  3.45  3.40  3.48  3.77  4.07  4.66  4.93   5.04   5.64   
2001-08-10  3.58  3.43  3.37  3.45  3.73  4.03  4.61  4.88   4.99   5.61   
2001-08-11  3.58  3.43  3.37  3.45  3.73  4.03  4.61  4.88   4.99   5.61   
2001-08-12  

In [3]:
#Create the real estate data function
def prepare_data_real_estate():
    CPI = pd.read_csv("Data/CPI.csv",index_col=0)
    CPI.columns = ["CPI"]
    CPI.index = pd.to_datetime(CPI.index)

    real_estate = pd.read_csv("Data/Real Estate.csv",index_col=0)
    real_estate.index = pd.to_datetime(real_estate.index)
    real_estate.columns = ["Real Estate"]

    multi_index = pd.MultiIndex.from_tuples(zip(real_estate.index.year,real_estate.index.quarter))
    real_estate.index = multi_index
    quarterly_CPI = CPI.groupby([CPI.index.year, CPI.index.quarter]).last()
    real_estate.index.names = ["Year", "Quarter"]
    quarterly_CPI.index.names = ["Year", "Quarter"]

    data = real_estate.join(quarterly_CPI)
    return data
real_estate_data = prepare_data_real_estate()
print(real_estate_data)

              Real Estate      CPI
Year Quarter                      
1975 1              59.77   52.800
     2              60.97   53.500
     3              61.18   54.600
     4              62.22   55.600
1976 1              62.90   56.000
     2              65.40   56.700
     3              66.60   57.600
     4              67.36   58.400
1977 1              69.44   59.600
     2              72.69   60.500
     3              74.40   61.300
     4              77.25   62.300
1978 1              79.60   63.400
     2              82.73   65.000
     3              85.22   66.500
     4              87.50   67.900
1979 1              91.40   69.900
     2              94.40   72.200
     3              96.38   74.400
     4              98.29   76.900
1980 1             100.00   80.100
     2             101.60   82.500
     3             104.51   83.900
     4             104.79   86.400
1981 1             105.37   88.600
     2             107.55   90.500
     3             1

In [4]:
#We are going to define the report as of a year and quarter
#First let's review how we could get the first day of the next quarter

#If we want the next quarter we can just add 1, except for 4 where we want that to become 1
#If we do the modul function we can achieve this
for quarter in range(1,5):
    print(quarter % 4 + 1)
print()

2
3
4
1



In [5]:
#Define a function that processes the data given the year/quarter we want to use as of
def process_data_yield_curve(yield_curve, year, quarter):
    yield_curve = yield_curve.copy()
    
    #Find the next quarter
    end_quarter = quarter % 4 + 1
    #Convert to the month
    end_month = (end_quarter-1) * 3 + 1
    #Find the end month which is the same unless the quarter is 4
    end_year = year
    if quarter == 4:
        end_year += 1
    #Finally create the datetime and subtract one day from it
    end_date = datetime(end_year, end_month, 1) - pd.Timedelta("1D")
    
    yield_curve = yield_curve.loc[:end_date]
    
    
    yield_table = create_yield_table(yield_curve, end_date)
    return yield_table
yield_table = process_data_yield_curve(yield_curve, 2019, 1)
print(yield_table)

       Current  1 Year Ago Curve  1 Year Change  5 Years Ago Curve  \
1 Mo      2.43              1.63           0.80               0.03   
3 Mo      2.40              1.73           0.67               0.05   
6 Mo      2.44              1.93           0.51               0.07   
1 Yr      2.40              2.09           0.31               0.13   
2 Yr      2.27              2.27           0.00               0.44   
3 Yr      2.21              2.39          -0.18               0.90   
5 Yr      2.23              2.56          -0.33               1.73   
7 Yr      2.31              2.68          -0.37               2.30   
10 Yr     2.41              2.74          -0.33               2.73   
20 Yr     2.63              2.85          -0.22               3.31   
30 Yr     2.81              2.97          -0.16               3.56   

       5 Year Change  
1 Mo            2.40  
3 Mo            2.35  
6 Mo            2.37  
1 Yr            2.27  
2 Yr            1.83  
3 Yr            1.31 

In [6]:
#Define a similar function for the real estate data

In [7]:
def process_data_real_estate(data, year, quarter):
    returns = data / data.loc[(1999,4)]
    returns = returns.loc[(2000,1):]
    returns["Real Estate Inflation Adjusted"] = returns["Real Estate"] / returns["CPI"]
    returns = returns.loc[:(year, quarter)]
    
    quarterly_returns = data["Real Estate"].pct_change()
    quarterly_returns = quarterly_returns.loc[(2000,1):(year, quarter)]
    quarterly_returns = quarterly_returns.unstack()
    
    quarterly_returns_adj = returns["Real Estate Inflation Adjusted"].pct_change()
    #Because there is no baseline to compare to, the percent change will be negative in the first year
    #We can set it to be equal to the index - 1
    quarterly_returns_adj.iloc[0] = returns["Real Estate Inflation Adjusted"].iloc[0] - 1
    quarterly_returns_adj = quarterly_returns_adj.unstack()
    
    return returns, quarterly_returns, quarterly_returns_adj
returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, 2019, 1)
print(returns_RE.head(5))
print()
print(quarterly_returns_RE.head(5))
print()
print(quarterly_returns_adj_RE.head(5))

              Real Estate       CPI  Real Estate Inflation Adjusted
Year Quarter                                                       
2000 1           1.019145  1.013033                        1.006033
     2           1.035752  1.020142                        1.015302
     3           1.054586  1.028436                        1.025427
     4           1.070837  1.034360                        1.035265
2001 1           1.097240  1.043246                        1.051755

Quarter         1         2         3         4
Year                                           
2000     0.019145  0.016295  0.018183  0.015410
2001     0.024656  0.016759  0.015525  0.012418
2002     0.014362  0.017029  0.019829  0.014168
2003     0.012187  0.012184  0.015411  0.028326
2004     0.015643  0.024409  0.038143  0.021346

Quarter         1         2         3         4
Year                                           
2000     0.006033  0.009213  0.009972  0.009594
2001     0.015928  0.007604  0.013244  0.0

In [8]:
#Let's take a step back and combine what we have created so far
def create_report(year, quarter):
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
create_report(2019, 1)

In [9]:
returns_RE.columns

Index(['Real Estate', 'CPI', 'Real Estate Inflation Adjusted'], dtype='object')

In [10]:
#Create a function for building and saving the images
def create_images(yield_table, returns_RE):
    graph_data = yield_table[['Current', '1 Year Ago Curve', '5 Years Ago Curve']].copy()
    ax = graph_data.plot(kind="line")
    current_lim = plt.ylim()
    new_lim = [min(current_lim[0],0),current_lim[1]]
    plt.ylim(new_lim)
    plt.xlabel("Maturity")
    plt.ylabel("Yield")
    plt.title("Yield Curves")
    ax.yaxis.set_major_formatter(PercentFormatter())
    plt.savefig("Images/Treasury Yield Curves.png")
    plt.close()
    
    ax = returns_RE[['Real Estate', 'CPI']].plot(kind='line')
    quarter_index = returns_RE.index.map(lambda x: str(x[0]) +"Q"+str(x[1]))
    index_positions = list(range(len(quarter_index)))
    ax.xaxis.set_ticks(index_positions[::4])
    ax.xaxis.set_ticklabels(quarter_index[::4])
    plt.xticks(rotation=70)
    plt.ylabel("Normalized Index Value")
    plt.xlabel("Date")
    plt.title("Normalized Indices")
    plt.savefig("Images/Real Estate vs. Inflation.png", bbox_inches="tight")
    plt.close()
    
    ax = returns_RE[["Real Estate", "Real Estate Inflation Adjusted"]].plot(kind='line')
    ax.xaxis.set_ticks(index_positions[::4])
    ax.xaxis.set_ticklabels(quarter_index[::4])
    plt.xticks(rotation=70)
    plt.xlabel("Quarter")
    plt.ylabel("Index Level")
    plt.title("Real Estate Nominal vs. Inflation Adjusted")
    plt.savefig("Images/Real Estate Inflation Adjusted.png", bbox_inches="tight")
    plt.close()
create_images(yield_table, returns_RE)

In [11]:
#Update the main function
def create_report(year, quarter):
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    create_images(yield_table, returns_RE)
create_report(2019, 1)

In [12]:
#Now, let's begin with writing the yield curve sheet

def create_report(year, quarter):
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    create_images(yield_table, returns_RE)
    
    wb = openpyxl.Workbook()
    ws = wb.active
    write_treasury_sheet(ws, yield_table)
    wb.save("Workbooks/Economic Report.xlsx")
create_report(2019, 1)

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:124: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).


In [13]:
#Now add in naming the active sheet as yield curve, creating a new sheet, writing the real estate data
#and naming it real estate

def create_report(year, quarter):
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    create_images(yield_table, returns_RE)
    
    wb = openpyxl.Workbook()
    ws = wb.active
    write_treasury_sheet(ws, yield_table)
    ws.title = "Yield Curve"
    ws = wb.create_sheet("Real Estate")
    create_real_estate_report(ws, quarterly_returns_RE, "Images/Real Estate vs. Inflation.png")
    wb.save("Workbooks/Economic Report.xlsx")
    
create_report(2019, 1)

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:124: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).


In [14]:
#Make it so we have two real estate sheets, one for nominal, one for inflation adjusted
#Also add in the date to the workbook title

def create_report(year, quarter):
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    create_images(yield_table, returns_RE)
    
    wb = openpyxl.Workbook()
    ws = wb.active
    write_treasury_sheet(ws, yield_table)
    ws.title = "Yield Curve"
    ws = wb.create_sheet("Real Estate Nominal")
    create_real_estate_report(ws, quarterly_returns_RE, "Images/Real Estate vs. Inflation.png")
    ws = wb.create_sheet("Real Estate Adjusted")
    create_real_estate_report(ws, quarterly_returns_adj_RE, "Images/Real Estate Inflation Adjusted.png")
    wb.save("Workbooks/Economic Report {}Q{}.xlsx".format(year, quarter))
    
create_report(2019, 1)

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:124: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).


In [15]:
#Finally, let's clean up the real estate worksheet building function to look nicer
def write_treasury_sheet(ws, yield_curve_table):
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_columns(ws,2,[15]*len(yield_curve_table.columns))
    
    #Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    #Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
    #Add borders
    create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])
    
    #Get rid of gridlines
    ws.sheet_view.showGridLines = False
    
    #Add yield curve graphs
    img = Image('Images/Treasury Yield Curves.png')
    ws.add_image(img, 'H1')
    
def create_real_estate_report(ws, real_estate_returns, image_name, title):
    #Write the table
    write_to_excel(ws, 4, 2, real_estate_returns)
    
    #Create the axis labels
    create_axis_labels(ws, real_estate_returns)
    
    #Create percentage number formatting
    create_number_formatting(ws, 5, 3, real_estate_returns.shape[0], real_estate_returns.shape[1],"0.0%")
    
    #Map the red-green color map
    red_green_map(ws, real_estate_returns, 5, 3)
    
    #Add image
    img = Image(image_name)
    ws.add_image(img, 'H5')
    
    #Bold the index and columns
    bold_cells_line(ws, 5, 2, len(real_estate_returns), vertical=True)
    bold_cells_line(ws, 4, 3, len(real_estate_returns.columns), vertical=False)
    
    #Add title
    create_title(ws, 'B1:F1', title)
    
    #Add borders
    create_border(ws, 5, 3, real_estate_returns.shape[0], real_estate_returns.shape[1])
    
    #Get rid of gridlines
    ws.sheet_view.showGridLines = False
    
def create_report(year, quarter):
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    create_images(yield_table, returns_RE)
    
    wb = openpyxl.Workbook()
    ws = wb.active
    write_treasury_sheet(ws, yield_table)
    ws.title = "Yield Curve"
    ws = wb.create_sheet("Real Estate Nominal")
    create_real_estate_report(ws, quarterly_returns_RE, "Images/Real Estate vs. Inflation.png",
                             "Nominal Real Estate Returns")
    ws = wb.create_sheet("Real Estate Adjusted")
    create_real_estate_report(ws, quarterly_returns_adj_RE, "Images/Real Estate Inflation Adjusted.png",
                             "Adjusted Real Estate Returns")
    wb.save("Workbooks/Economic Report {}Q{}.xlsx".format(year, quarter))
    
create_report(2019, 1)

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:124: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).


In [16]:
#Now run a few times and see how it automatically creates them!
create_report(2019, 4)
create_report(2018, 2)
create_report(2017, 4)

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:124: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
